# Inconsistent Data Entry

## 0. Modules

In [1]:
import numpy as np
import pandas as pd

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

# read in all our data
professors = pd.read_csv("pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

## 1. Do some preliminary text pre-processing

首先我們快速查看數據的前幾行。

In [2]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


假設我們有興趣清理「國家」欄位，以確保其中沒有資料輸入的不一致。  
當然，我們可以逐行檢查，並在發現不一致時手動更正。  
不過，有一種更有效的方法可以做到這一點。

In [3]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

單從這點來看，我就能發現一些由於資料輸入不一致而造成的問題，  
例如「 Germany」和「germany」，或是「 New Zealand」和「New Zealand」。

我要做的第一件事是將所有內容轉換為小寫（如果我希望，最後可以改回來）並刪除單元格開頭和結尾的任何空白。  
在文本資料中，大小寫不一致和尾隨空白非常常見，通過這樣做，你可以修正大約 80% 的文本資料輸入不一致的問題。

In [4]:
# convert to lower case
professors['Country'] = professors['Country'].str.lower()
# remove trailing white spaces
professors['Country'] = professors['Country'].str.strip()

接下來，我們將處理更困難的不一致性問題。

## 2. Use fuzzy matching to correct inconsistent data entry

好的，讓我們再次檢視「國家」欄位，看看是否還有更多的數據清理工作需要進行。

In [5]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

看起來確實還有另一個不一致的地方：「southkorea」和「south korea」應該是相同的。

我們將使用 fuzzywuzzy 套件來幫助識別哪些字串彼此最接近。這個資料集足夠小，我們可能可以手動更正錯誤，  
但這種方法並不適合大規模應用。  
（你會想要手動更正一千個錯誤嗎？那麼一萬個錯誤呢？盡早自動化處理通常是個好主意。而且，這很有趣！）

**模糊匹配(Fuzzy matching)：自動尋找與目標字串非常相似的文字字串的過程。**  
一般來說，一個字串被認為與另一個字串「更接近」的程度取決於如果你要將一個字串變換為另一個字串，  
需要更改的字元數量越少。所以「apple」和「snapple」之間有兩個變化（增加「s」和「n」），  
而「in」和「on」之間有一個變化（將「i」替換為「o」）。  
你不會總是能夠100%依賴模糊匹配，但它通常會至少節省你一些時間。

Fuzzywuzzy 返回兩個字串之間的比率。比率越接近 100，兩個字串之間的編輯距離就越小。  
在這裡，我們將從我們的城市列表中獲得與「south korea」距離最近的十個字串。

In [6]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('austria', 33),
 ('ireland', 33),
 ('pakistan', 32),
 ('portugal', 32),
 ('scotland', 32),
 ('australia', 30)]

我們可以看到在城市列表中有兩個項目與「south korea」非常接近：「south korea」和「southkorea」。  
讓我們將「國家」欄中所有比率大於 47 的行替換為「south korea」。

為了做到這一點，我將編寫一個函數。（如果你認為可能需要多次執行某個特定任務，編寫一個通用的函數以便重複使用是個好主意。  
這可以避免你過於頻繁地複製和粘貼代碼，從而節省時間並有助於防止錯誤。）

In [7]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

既然我們已經有了一個函數，現在可以對它進行測試了！

In [8]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


現在，讓我們再次檢查我們「國家」欄中的唯一值，確保我們正確整理了「south korea」。

In [9]:
# get all the unique values in the 'Country' column
countries = professors['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea', 'spain',
       'sweden', 'thailand', 'turkey', 'uk', 'urbana', 'usa', 'usofa'],
      dtype=object)

太好了！現在我們的資料框中只有「south korea」，而且我們不需要手動更改任何內容。